In [1]:
import pandas as pd
import csv
import gensim
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from scipy import spatial
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from collections import Counter
import gc
import math
gc.collect()

9

In [2]:
df = pd.read_csv("FINAL_DATASET.csv")
df.shape

(324, 21)

In [3]:
df.head()

,Movie_Number,Movie_Name,Year_of_Release,IMDB_Rating,Number_of_Votes,Genres,rating,year,id_x,imdbid,...,crew,id_y,budget,genres,popularity,production_companies,revenue,title,vote_count,ROI
0,m0,10 things i hate about you,1999,6.9,62847,"['comedy', 'romance']",3,1999,374,147800.0,...,"[{'credit_id': '52fe43e6c3a36847f807731d', 'de...",4951,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",12.069966,"[{'name': 'Mad Chance', 'id': 1757}, {'name': ...",53478166.0,10 Things I Hate About You,1768.0,3.342385
1,m3,2001: a space odyssey,1968,8.4,163227,"['adventure', 'mystery', 'sci-fi']",0,1968,919,62622.0,...,"[{'credit_id': '52fe4212c3a36847f8001a0b', 'de...",62,10500000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",22.494622,"[{'name': 'Stanley Kubrick Productions', 'id':...",68700000.0,2001: A Space Odyssey,3075.0,6.542857
2,m4,48 hrs.,1982,6.9,22289,"['action', 'comedy', 'crime', 'drama', 'thrill...",2,1982,5355,83511.0,...,"[{'credit_id': '52fe421fc3a36847f8005927', 'de...",150,12000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",15.297121,"[{'name': 'Paramount Pictures', 'id': 4}]",78868508.0,48 Hrs.,364.0,6.572376
3,m6,8mm,1999,6.3,48212,"['crime', 'mystery', 'thriller']",1,1999,146,134273.0,...,"[{'credit_id': '52fe4498c3a36847f809f18b', 'de...",8224,40000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 80, 'n...",7.473718,"[{'name': 'Columbia Pictures Corporation', 'id...",96618699.0,8MM,479.0,2.415467
4,m7,a nightmare on elm street 4: the dream master,1988,5.2,13590,"['fantasy', 'horror', 'thriller']",3,1988,6020,95742.0,...,"[{'credit_id': '52fe43309251416c75006d1d', 'de...",10131,7000000,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",8.120317,"[{'name': 'New Line Cinema', 'id': 12}]",72500000.0,A Nightmare on Elm Street 4: The Dream Master,287.0,10.357143


In [4]:
analysis_df = pd.DataFrame(columns = ['Year',
                                      'Movie_Title', 
                                      '#_of_Female_Cast', 
                                      '#_of_Male_Cast',
                                      '#_of_Female_Dialogue',
                                      '#_of_Male_Dialogue',
                                      '#_of_Female_Crew',
                                     '#_of_Male_Crew'])

In [5]:
analysis_df['Movie_Title'] = df['Movie_Name']
analysis_df['Year'] = df['Year_of_Release']
analysis_df.shape

(324, 8)

In [6]:
analysis_df.head()

,Year,Movie_Title,#_of_Female_Cast,#_of_Male_Cast,#_of_Female_Dialogue,#_of_Male_Dialogue,#_of_Female_Crew,#_of_Male_Crew
0,1999,10 things i hate about you,NaN,NaN,NaN,NaN,NaN,NaN
1,1968,2001: a space odyssey,NaN,NaN,NaN,NaN,NaN,NaN
2,1982,48 hrs.,NaN,NaN,NaN,NaN,NaN,NaN
3,1999,8mm,NaN,NaN,NaN,NaN,NaN,NaN
4,1988,a nightmare on elm street 4: the dream master,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
mcm_df = pd.read_csv("mcm_csv.csv")

In [8]:
mcm_df.loc[mcm_df["Movie_Number"] == "m0"]

,Character_Id,Character_Name,Movie_Number,Movie_Title,Gender,Position_in_Credits
0,u0,BIANCA,m0,10 things i hate about you,f,4
1,u1,BRUCE,m0,10 things i hate about you,m,?
2,u2,CAMERON,m0,10 things i hate about you,m,3
3,u3,CHASTITY,m0,10 things i hate about you,f,?
4,u4,JOEY,m0,10 things i hate about you,m,6
5,u5,KAT,m0,10 things i hate about you,f,2
6,u6,MANDELLA,m0,10 things i hate about you,f,7
7,u7,MICHAEL,m0,10 things i hate about you,m,5
8,u8,MISS PERKY,m0,10 things i hate about you,m,?
9,u9,PATRICK,m0,10 things i hate about you,m,1


In [9]:
## Getting the number of female and male characters
for rows in analysis_df.itertuples():
    f_count = 0
    m_count = 0
    for row2 in mcm_df.itertuples():
        if row2[4] == rows[2]:
            if row2[5] == 'f':
                f_count += 1
            else:
                m_count += 1
    analysis_df.at[rows.Index, "#_of_Female_Cast"] = f_count
    analysis_df.at[rows.Index, "#_of_Male_Cast"] = m_count    

In [10]:
analysis_df.head()

,Year,Movie_Title,#_of_Female_Cast,#_of_Male_Cast,#_of_Female_Dialogue,#_of_Male_Dialogue,#_of_Female_Crew,#_of_Male_Crew
0,1999,10 things i hate about you,5,7,NaN,NaN,NaN,NaN
1,1968,2001: a space odyssey,2,11,NaN,NaN,NaN,NaN
2,1982,48 hrs.,7,9,NaN,NaN,NaN,NaN
3,1999,8mm,2,13,NaN,NaN,NaN,NaN
4,1988,a nightmare on elm street 4: the dream master,6,2,NaN,NaN,NaN,NaN


In [11]:
del mcm_df

In [12]:
crew_df = pd.read_csv("Crew_Dataset.csv")

In [13]:
crew_df.loc[crew_df["Movie_Number"] == "m0"]

,Movie_Number,Movie_Name,Crew_Name,Job,Gender
0,m0,10 things i hate about you,Charles Graffeo,Set Decoration,m
1,m0,10 things i hate about you,William Shakespeare,Theatre Play,m
2,m0,10 things i hate about you,Mark Irwin,Director of Photography,m
3,m0,10 things i hate about you,O. Nicholas Brown,Editor,m
4,m0,10 things i hate about you,Kimberly A. Tillman,Costume Design,f
5,m0,10 things i hate about you,Richard Gibbs,Original Music Composer,m
6,m0,10 things i hate about you,Andrew Lazar,Producer,m
7,m0,10 things i hate about you,Carol Winstead Wood,Production Design,f
8,m0,10 things i hate about you,Gil Junger,Director,m
9,m0,10 things i hate about you,Karen McCullah Lutz,Writer,f


In [14]:
## Getting the number of female and male characters
for rows in analysis_df.itertuples():
    f_count = 0
    m_count = 0
    for row2 in crew_df.itertuples():
        if row2[2] == rows[2]:
            if row2[5] == 'f':
                f_count += 1
            else:
                m_count += 1
    analysis_df.at[rows.Index, "#_of_Female_Crew"] = f_count
    analysis_df.at[rows.Index, "#_of_Male_Crew"] = m_count   

In [15]:
del crew_df
analysis_df.head()

,Year,Movie_Title,#_of_Female_Cast,#_of_Male_Cast,#_of_Female_Dialogue,#_of_Male_Dialogue,#_of_Female_Crew,#_of_Male_Crew
0,1999,10 things i hate about you,5,7,NaN,NaN,11,26
1,1968,2001: a space odyssey,2,11,NaN,NaN,2,17
2,1982,48 hrs.,7,9,NaN,NaN,1,14
3,1999,8mm,2,13,NaN,NaN,15,62
4,1988,a nightmare on elm street 4: the dream master,6,2,NaN,NaN,3,12


In [16]:
## Getting the number of dialogues
ml_df = pd.read_csv("ml_csv.csv")
mcm_df = pd.read_csv("mcm_csv.csv")
print(len(ml_df.Line_Id.unique()), ml_df.shape)

304713 (304713, 5)


In [17]:
ml_df.head(1)

,Line_Id,Character_Id,Movie_Number,Character_Name,Dialogue
0,L1045,u0,m0,BIANCA,They do not!


In [18]:
mcm_df.head(1)

,Character_Id,Character_Name,Movie_Number,Movie_Title,Gender,Position_in_Credits
0,u0,BIANCA,m0,10 things i hate about you,f,4


In [19]:
new_df = pd.merge(ml_df, mcm_df, how='left')
new_df.drop_duplicates(keep='first', inplace=True)
new_df.shape

(304713, 8)

In [20]:
new_df.head()

,Line_Id,Character_Id,Movie_Number,Character_Name,Dialogue,Movie_Title,Gender,Position_in_Credits
0,L1045,u0,m0,BIANCA,They do not!,10 things i hate about you,f,4
1,L1044,u2,m0,CAMERON,They do to!,10 things i hate about you,m,3
2,L985,u0,m0,BIANCA,I hope so.,10 things i hate about you,f,4
3,L984,u2,m0,CAMERON,She okay?,10 things i hate about you,m,3
4,L925,u0,m0,BIANCA,Let's go.,10 things i hate about you,f,4


In [21]:
## Getting the number of female and male characters
for rows in analysis_df.itertuples():
    f_count = 0
    m_count = 0
    for row2 in new_df.itertuples():
        if row2[6] == rows[2]:
            if row2[7] == 'f':
                f_count += 1
            else:
                m_count += 1
    analysis_df.at[rows.Index, "#_of_Female_Dialogue"] = f_count
    analysis_df.at[rows.Index, "#_of_Male_Dialogue"] = m_count  

In [22]:
del new_df
del ml_df
del mcm_df

In [23]:
analysis_df.head()

,Year,Movie_Title,#_of_Female_Cast,#_of_Male_Cast,#_of_Female_Dialogue,#_of_Male_Dialogue,#_of_Female_Crew,#_of_Male_Crew
0,1999,10 things i hate about you,5,7,295,374,11,26
1,1968,2001: a space odyssey,2,11,17,255,2,17
2,1982,48 hrs.,7,9,109,469,1,14
3,1999,8mm,2,13,37,547,15,62
4,1988,a nightmare on elm street 4: the dream master,6,2,95,60,3,12


In [24]:
analysis_df.to_csv("Analysis_csv.csv", index = False)

# Getting Sentiment scores in place

In [25]:
analy = pd.read_csv("Analysis_csv.csv")
print(analy.shape)

(324, 8)


In [26]:
analy.head()

,Year,Movie_Title,#_of_Female_Cast,#_of_Male_Cast,#_of_Female_Dialogue,#_of_Male_Dialogue,#_of_Female_Crew,#_of_Male_Crew
0,1999,10 things i hate about you,5,7,295,374,11,26
1,1968,2001: a space odyssey,2,11,17,255,2,17
2,1982,48 hrs.,7,9,109,469,1,14
3,1999,8mm,2,13,37,547,15,62
4,1988,a nightmare on elm street 4: the dream master,6,2,95,60,3,12


In [27]:
ml = pd.read_csv("ml_csv.csv")
mcm = pd.read_csv("mcm_csv.csv")
print(ml.shape)

(304713, 5)


In [28]:
ml.loc[ml["Movie_Number"] == 'm0'].shape

(669, 5)

In [29]:
ml.head()

,Line_Id,Character_Id,Movie_Number,Character_Name,Dialogue
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [30]:
mcm.head()

,Character_Id,Character_Name,Movie_Number,Movie_Title,Gender,Position_in_Credits
0,u0,BIANCA,m0,10 things i hate about you,f,4
1,u1,BRUCE,m0,10 things i hate about you,m,?
2,u2,CAMERON,m0,10 things i hate about you,m,3
3,u3,CHASTITY,m0,10 things i hate about you,f,?
4,u4,JOEY,m0,10 things i hate about you,m,6


In [31]:
ml_mcm = pd.merge(ml, mcm, on = ["Movie_Number", "Character_Name"], how = "left")
ml_mcm.shape

(304713, 9)

In [32]:
ml_mcm.head()

,Line_Id,Character_Id_x,Movie_Number,Character_Name,Dialogue,Character_Id_y,Movie_Title,Gender,Position_in_Credits
0,L1045,u0,m0,BIANCA,They do not!,u0,10 things i hate about you,f,4
1,L1044,u2,m0,CAMERON,They do to!,u2,10 things i hate about you,m,3
2,L985,u0,m0,BIANCA,I hope so.,u0,10 things i hate about you,f,4
3,L984,u2,m0,CAMERON,She okay?,u2,10 things i hate about you,m,3
4,L925,u0,m0,BIANCA,Let's go.,u0,10 things i hate about you,f,4


In [33]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import numpy as np

In [34]:
sid = SentimentIntensityAnalyzer()

In [35]:
# Now doing it for the movie
for rows in analy.itertuples():
    sent_dict = {}
    for rows2 in ml_mcm.itertuples():
        if rows2[7] == rows[2]:
            l = rows2[5]
            line = re.sub(r'\.\.+', '', str(l))
            s_score = sid.polarity_scores(line)
            sent_dict[rows2.Index] = {rows2[8] : s_score.get('compound')}
#     print(sent_dict, len(sent_dict))
    g_listm = []
    genre_listm = []
    g_listf = []
    genre_listf = []
    sum_f = 0; sum_m = 0
    for key, vals in sent_dict.items():
        gender = str(list(sent_dict.get(key, {}).keys())).replace('[','').replace(']', '').replace('\'', '')
        genre = str(list(sent_dict.get(key, {}).values())).replace('[','').replace(']', '').replace('\'', '')
        if gender == 'm' or gender == 'M':
            g_listm.append(gender)
            genre_listm.append(genre)
        else:
            g_listf.append(gender)
            genre_listf.append(genre)
    genre_listf = np.array(genre_listf, dtype = float)
    genre_listm = np.array(genre_listm, dtype = float)
#     print(genre_listf,  "\n----\n", genre_listm)
    try:
        sum_f = sum(genre_listf)/len(genre_listf)
    except:
        sum_f = 0
    try:
        sum_m = sum(genre_listm)/len(genre_listm)
    except:
        sum_m = 0
    print(sum_f, "----", sum_m)
    analy.at[rows.Index, "Sentiment_Female"] = sum_f
    analy.at[rows.Index, "Sentiment_Male"] = sum_m

0.027764406779661036 ---- 0.02628743315508023
0.24472352941176467 ---- 0.14193333333333336
-0.026442201834862386 ---- 0.0028660980810234623
-0.10827837837837839 ---- 0.011065265082266927
-0.058179999999999996 ---- -0.052153333333333336
0.021137864077669904 ---- 0.017488301886792467
0.09267380952380952 ---- 0.06590328947368423
0.11727142857142857 ---- 0.0332795847750865
0.0014864406779661007 ---- 0.04003021978021977
0.04253575757575758 ---- 0.1218619952494063
0.01419607843137255 ---- -0.022654117647058825
0.04387714285714286 ---- -0.0016661101836394
0.07257290322580644 ---- 0.03607648183556403
0.015314503816793898 ---- 0.07552151898734176
0.1005069565217391 ---- 0.05452418478260873
0.14561111111111108 ---- 0.08193978494623659
0.17236323529411765 ---- 0.18250740740740728
-0.02928759124087592 ---- -0.07126635859519406
0.08284462809917355 ---- 0.08067237762237765
0.010062222222222212 ---- 0.047314206128133716
0.0554740524781341 ---- 0.10653218210361061
0.08315777027027024 ---- 0.1266816793

0.007757142857142853 ---- 0.009626978417266209
-0.040465185185185173 ---- -0.04038037383177571
-0.0036949494949494906 ---- -0.023687529976019188
-0.00777708333333333 ---- 0.0472375
0.09432749999999998 ---- 0.06388349673202612
0.042046666666666684 ---- 0.063089485458613
0.0875187919463087 ---- 0.15413799126637556
0.08650886075949371 ---- -0.011226989619377178
0.019527536231884076 ---- 0.009348578491965397
0.022190389016018316 ---- 0.04484222972972972
0.10978125 ---- 0.049861764705882355
0.0025000000000000005 ---- -0.028413302752293576
-0.0016075268817204369 ---- 0.021094881889763793
0.11667500000000003 ---- -0.003003107344632764
0.08229343434343435 ---- 0.060894901960784314
-0.008597580645161293 ---- -0.01330610687022901
-0.06495364238410593 ---- -0.07437692307692306
-0.06641400000000001 ---- -0.04826516853932586
-0.04989999999999999 ---- 0.0042691406250000025
0.10477008928571431 ---- 0.05440457746478875
0.06772298136645964 ---- 0.04141899641577062
0.020293333333333344 ---- 0.0172105769

In [36]:
analy.tail()

,Year,Movie_Title,#_of_Female_Cast,#_of_Male_Cast,#_of_Female_Dialogue,#_of_Male_Dialogue,#_of_Female_Crew,#_of_Male_Crew,Sentiment_Female,Sentiment_Male
319,1985,witness,2,8,119,238,0,7,0.057887,-0.022337
320,2009,watchmen,4,12,115,252,6,18,0.002022,-0.037667
321,2002,xxx,1,10,5,407,37,102,-0.002368,-0.010369
322,2000,x-men,2,7,39,136,3,20,-0.017695,0.015544
323,1974,young frankenstein,3,11,121,455,9,35,0.182547,0.086947


In [37]:
analy.to_csv("Analysis_csv.csv", index = False)

# Code for the Genre tagging

In [38]:
import gensim
try:
    model = gensim.models.KeyedVectors.load_word2vec_format('../Google_Billion_Word_Corpus/GoogleNews-vectors-negative300.bin', binary=True)
except Exception as e:
    print(e)

In [39]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [40]:
ml = pd.read_csv("ml_csv.csv")
mcm = pd.read_csv("mcm_csv.csv")
print(ml.shape)

(304713, 5)


In [41]:
ml_mcm = pd.merge(ml, mcm, on = ["Movie_Number", "Character_Name"], how = "left")
ml_mcm.shape

(304713, 9)

In [42]:
ml_mcm.head()

,Line_Id,Character_Id_x,Movie_Number,Character_Name,Dialogue,Character_Id_y,Movie_Title,Gender,Position_in_Credits
0,L1045,u0,m0,BIANCA,They do not!,u0,10 things i hate about you,f,4
1,L1044,u2,m0,CAMERON,They do to!,u2,10 things i hate about you,m,3
2,L985,u0,m0,BIANCA,I hope so.,u0,10 things i hate about you,f,4
3,L984,u2,m0,CAMERON,She okay?,u2,10 things i hate about you,m,3
4,L925,u0,m0,BIANCA,Let's go.,u0,10 things i hate about you,f,4


In [43]:
mtm_df = pd.read_csv("mtm_csv.csv")
analy = pd.read_csv("Analysis_csv.csv")
print(analy.shape)
mtm_df.rename(columns = {'Movie_Name': 'Movie_Title'}, inplace = True)
mtm_df.head()

(324, 10)


,Movie_Number,Movie_Title,Year_of_Release,IMDB_Rating,Number_of_Votes,Genres
0,m0,10 things i hate about you,1999,6.9,62847,"['comedy', 'romance']"
1,m1,1492: conquest of paradise,1992,6.2,10421,"['adventure', 'biography', 'drama', 'history']"
2,m2,15 minutes,2001,6.1,25854,"['action', 'crime', 'drama', 'thriller']"
3,m3,2001: a space odyssey,1968,8.4,163227,"['adventure', 'mystery', 'sci-fi']"
4,m4,48 hrs.,1982,6.9,22289,"['action', 'comedy', 'crime', 'drama', 'thrill..."


In [44]:
analy.head(2)

,Year,Movie_Title,#_of_Female_Cast,#_of_Male_Cast,#_of_Female_Dialogue,#_of_Male_Dialogue,#_of_Female_Crew,#_of_Male_Crew,Sentiment_Female,Sentiment_Male
0,1999,10 things i hate about you,5,7,295,374,11,26,0.027764,0.026287
1,1968,2001: a space odyssey,2,11,17,255,2,17,0.244724,0.141933


In [45]:
movie_list = analy.Movie_Title.unique()
print(len(movie_list))

324


In [46]:
new_Df = ml_mcm.loc[ml_mcm['Movie_Title'].isin(movie_list)]

In [47]:
new_Df.loc[new_Df['Movie_Number'] == 'm0'].shape

(669, 9)

In [48]:
new_Df.shape

(163017, 9)

In [49]:
new_Df['Dialogue'] = new_Df['Dialogue'].str.lower()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [50]:
new_Df.head(1)

,Line_Id,Character_Id_x,Movie_Number,Character_Name,Dialogue,Character_Id_y,Movie_Title,Gender,Position_in_Credits
0,L1045,u0,m0,BIANCA,they do not!,u0,10 things i hate about you,f,4


In [51]:
analy = pd.merge(analy, mtm_df, on = 'Movie_Title', how = 'left')
analy.drop(['Movie_Number', 'Year_of_Release', 'IMDB_Rating', 'Number_of_Votes'], axis = 1, inplace = True)
print(analy.shape)

(324, 11)


In [52]:
analy.head(2)

,Year,Movie_Title,#_of_Female_Cast,#_of_Male_Cast,#_of_Female_Dialogue,#_of_Male_Dialogue,#_of_Female_Crew,#_of_Male_Crew,Sentiment_Female,Sentiment_Male,Genres
0,1999,10 things i hate about you,5,7,295,374,11,26,0.027764,0.026287,"['comedy', 'romance']"
1,1968,2001: a space odyssey,2,11,17,255,2,17,0.244724,0.141933,"['adventure', 'mystery', 'sci-fi']"


In [53]:
tokenizer = RegexpTokenizer(r'\w+')

for rows in analy.itertuples():
    gdict = {}
    # Genre information
    j = rows[11]
    j=j.replace('[',"")
    j=j.replace(']',"")
    j=j.replace("'","")
    j=j.split(", ")
    j = ['science' if x == 'sci-fi' else x for x in j]
    print(j)
    try:
        j = [j[0], j[1], j[2]]
    except:
        if len(j) == 2:
            j = [j[0], j[1]]
        else:
            j = [j[0]]
    
    j = pd.Series(j).drop_duplicates().tolist()    
    
    try:
        print(j[0], j[1], j[2])
    except:
        if len(j) == 2:
            print(j[0], j[1])
        else:
            print(j[0])
        
    gdict[j[0]] = model[j[0]]
    try:
        gdict[j[1]] = model[j[1]]
    except:
        gdict["Nan"] = np.zeros(300)
    try:
        gdict[j[2]] = model[j[2]]
    except:
        gdict["NaN"] = np.zeros(300)
        
    women_vals = [0, 0, 0]
    men_vals = [0, 0, 0]
    fc = 0
    mc = 0
    final_vals = []
    
    for rows2 in new_Df.itertuples():
        if rows2[7] == rows[2]:
            try:
                line = tokenizer.tokenize(rows2[5])
            except:
                line=''
            av = np.zeros(300)
            val2={}
            
            line=[w for w in line if not w in stop_words]
            l1=len(line)
            for tokens in line:
                try:
                    av+=model[tokens]
                except:
                    l1-=1
                    pass
            av=av/l1
            # Cosine similarities
            for key, vals in gdict.items():
                try:
                    val2[key] = 1 - spatial.distance.cosine(av, vals)
                except:
                    pass
            final_vals = [val2[k] for k in val2]
            final_vals = [0 if math.isnan(x) else x for x in final_vals]
#             print(final_vals, rows2[8])
            
            if rows2[8] == 'f' or rows2[8] == 'F':
                women_vals[0] += final_vals[0]
                women_vals[1] += final_vals[1]
                women_vals[2] += final_vals[2]
                fc += 1
            else:
                men_vals[0] += final_vals[0]
                men_vals[1] += final_vals[1]
                men_vals[2] += final_vals[2]
                mc += 1
    
#     print(women_vals[0]/fc, men_vals[0]/mc, women_vals[1]/fc, men_vals[1]/mc)
    try:
        analy.at[rows.Index, "Genre1_Tag_Female"] = float(women_vals[0]/fc)
    except:
        analy.at[rows.Index, "Genre1_Tag_Female"] = float(0)
    try:
        analy.at[rows.Index, "Genre1_Tag_Male"] = float(men_vals[0]/mc)
    except:
        analy.at[rows.Index, "Genre1_Tag_Male"] = float(0)
    try:
        analy.at[rows.Index, "Genre2_Tag_Female"] = float(women_vals[1]/fc)
    except:
        analy.at[rows.Index, "Genre2_Tag_Female"] = float(0)
    try:
        analy.at[rows.Index, "Genre2_Tag_Male"] = float(men_vals[1]/mc)
    except:
        analy.at[rows.Index, "Genre2_Tag_Male"] = float(0)
    try:
        analy.at[rows.Index, "Genre3_Tag_Female"] = float(women_vals[2]/fc)
    except:
        analy.at[rows.Index, "Genre3_Tag_Female"] = float(0)
    try:
        analy.at[rows.Index, "Genre3_Tag_Male"] = float(men_vals[2]/mc)
    except:
        analy.at[rows.Index, "Genre3_Tag_Male"] = float(0)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:64: RuntimeWarning:

invalid value encountered in true_divide

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/scipy/spatial/distance.py:698: RuntimeWarning:

invalid value encountered in double_scalars



['comedy', 'romance']
comedy romance
['adventure', 'mystery', 'science']
adventure mystery science
['action', 'comedy', 'crime', 'drama', 'thriller']
action comedy crime
['crime', 'mystery', 'thriller']
crime mystery thriller
['fantasy', 'horror', 'thriller']
fantasy horror thriller
['action', 'drama', 'thriller']
action drama thriller
['comedy', 'romance']
comedy romance
['crime', 'drama', 'science', 'thriller']
crime drama science
['action', 'science', 'thriller']
action science thriller
['biography', 'drama', 'music']
biography drama music
['horror', 'romance']
horror romance
['action', 'western']
action western
['drama', 'thriller']
drama thriller
['drama', 'thriller']
drama thriller
['action', 'adventure', 'comedy', 'crime']
action adventure comedy
['comedy', 'romance']
comedy romance
['drama', 'romance', 'war']
drama romance war
['crime', 'drama', 'mystery', 'thriller']
crime drama mystery
['adventure', 'drama', 'fantasy']
adventure drama fantasy
['drama', 'science', 'thriller']


['drama', 'history', 'war']
drama history war
['drama', 'romance', 'thriller']
drama romance thriller
['crime', 'drama', 'thriller']
crime drama thriller
['drama', 'fantasy', 'science']
drama fantasy science
['comedy', 'drama', 'romance']
comedy drama romance
['comedy', 'horror']
comedy horror
['horror', 'comedy', 'action']
horror comedy action
['biography', 'crime', 'drama']
biography crime drama
['action', 'crime', 'fantasy']
action crime fantasy
['action', 'crime', 'thriller']
action crime thriller
['comedy', 'mystery', 'romance', 'thriller']
comedy mystery romance
['horror', 'mystery', 'thriller']
horror mystery thriller
['crime', 'drama', 'mystery', 'thriller']
crime drama mystery
['drama', 'mystery']
drama mystery
['action', 'adventure', 'thriller']
action adventure thriller
['crime', 'drama', 'science']
crime drama science
['crime', 'drama', 'thriller']
crime drama thriller
['action', 'crime', 'mystery', 'romance', 'thriller']
action crime mystery
['drama', 'mystery', 'science',

['crime', 'romance', 'thriller']
crime romance thriller
['mystery', 'science', 'thriller']
mystery science thriller
['drama', 'fantasy', 'mystery', 'thriller']
drama fantasy mystery
['drama', 'western']
drama western
['crime', 'mystery', 'romance', 'thriller']
crime mystery romance
['drama', 'horror', 'mystery', 'thriller']
drama horror mystery
['comedy', 'fantasy', 'romance']
comedy fantasy romance
['crime', 'mystery', 'thriller']
crime mystery thriller
['action', 'western', 'comedy', 'science']
action western comedy
['action', 'adventure', 'drama', 'fantasy', 'romance']
action adventure drama
['drama', 'romance', 'thriller']
drama romance thriller
['action', 'crime', 'fantasy', 'mystery', 'science', 'thriller']
action crime fantasy
['action', 'adventure', 'crime']
action adventure crime
['action', 'science']
action science
['comedy', 'science']
comedy science


In [54]:
analy.tail(10)

,Year,Movie_Title,#_of_Female_Cast,#_of_Male_Cast,#_of_Female_Dialogue,#_of_Male_Dialogue,#_of_Female_Crew,#_of_Male_Crew,Sentiment_Female,Sentiment_Male,Genres,Genre1_Tag_Female,Genre1_Tag_Male,Genre2_Tag_Female,Genre2_Tag_Male,Genre3_Tag_Female,Genre3_Tag_Male
314,2000,what lies beneath,5,5,436,223,22,89,0.025850,0.003195,"['drama', 'horror', 'mystery', 'thriller']",0.083183,0.087377,0.122338,0.118682,0.120105,0.126256
315,2000,what women want,1,1,3,3,24,71,-0.119833,0.044167,"['comedy', 'fantasy', 'romance']",0.079808,0.151906,0.094969,0.181789,0.087453,0.145327
316,1998,wild things,6,7,245,264,1,8,-0.003185,-0.056314,"['crime', 'mystery', 'thriller']",0.125601,0.142793,0.135009,0.138269,0.070724,0.075105
317,1999,wild wild west,1,8,21,301,1,11,0.093195,0.083939,"['action', 'western', 'comedy', 'sci-fi']",0.127692,0.112407,0.117143,0.109587,0.106009,0.107918
318,1988,willow,3,7,65,233,11,45,-0.019435,0.035591,"['action', 'adventure', 'drama', 'fantasy', 'r...",0.080072,0.089630,0.117216,0.125480,0.099258,0.096071
319,1985,witness,2,8,119,238,0,7,0.057887,-0.022337,"['drama', 'romance', 'thriller']",0.084969,0.086732,0.112467,0.110036,0.050484,0.061333
320,2009,watchmen,4,12,115,252,6,18,0.002022,-0.037667,"['action', 'crime', 'fantasy', 'mystery', 'sci...",0.093158,0.107956,0.123250,0.141138,0.162767,0.165617
321,2002,xxx,1,10,5,407,37,102,-0.002368,-0.010369,"['action', 'adventure', 'crime']",0.126709,0.129786,0.146824,0.147584,0.138890,0.152288
322,2000,x-men,2,7,39,136,3,20,-0.017695,0.015544,"['action', 'sci-fi']",0.102302,0.108213,0.134645,0.115685,0.000000,0.000000
323,1974,young frankenstein,3,11,121,455,9,35,0.182547,0.086947,"['comedy', 'sci-fi']",0.107888,0.094795,0.096118,0.100548,0.000000,0.000000


In [55]:
gc.collect()

529

In [56]:
analy.drop(['Genres'], axis = 1, inplace = True)

In [57]:
analy.head()

,Year,Movie_Title,#_of_Female_Cast,#_of_Male_Cast,#_of_Female_Dialogue,#_of_Male_Dialogue,#_of_Female_Crew,#_of_Male_Crew,Sentiment_Female,Sentiment_Male,Genre1_Tag_Female,Genre1_Tag_Male,Genre2_Tag_Female,Genre2_Tag_Male,Genre3_Tag_Female,Genre3_Tag_Male
0,1999,10 things i hate about you,5,7,295,374,11,26,0.027764,0.026287,0.099730,0.114050,0.127356,0.130247,0.000000,0.000000
1,1968,2001: a space odyssey,2,11,17,255,2,17,0.244724,0.141933,0.140718,0.107436,0.126419,0.143354,0.086336,0.109146
2,1982,48 hrs.,7,9,109,469,1,14,-0.026442,0.002866,0.109943,0.106253,0.121081,0.130147,0.137317,0.138767
3,1999,8mm,2,13,37,547,15,62,-0.108278,0.011065,0.142303,0.135797,0.129424,0.144191,0.059458,0.074539
4,1988,a nightmare on elm street 4: the dream master,6,2,95,60,3,12,-0.058180,-0.052153,0.178471,0.189962,0.173313,0.171728,0.087794,0.091161


In [58]:
analy.to_csv("Analysis_csv.csv", index = False)